# A notebook with the FRB/US model
This notebook is based on the the FRB/US model packages located here (https://www.federalreserve.gov/econres/us-models-package.htm). The packages provides an Eviews (https://www.eviews.com/) implementation. The Eviews modelspecification has been transformed to ModelFlow BusinessLogic specification. 

FRB/US comes in two flawors: **VAR** in which expectaions are generated by backward looking equations, and **MCE** in which expectation are based on leaded values. 
This notebook implement the **VAR** flavour. 

The tool used is highly experimental and the results are purely for illustration. 

You can run each cell by pressing the run tool or shift+enter
To run all cells use select cell/run all in the menu bar. 

# Import Python libraries

In [ ]:
%matplotlib inline

import pandas as pd
from numba import jit

# point to the modelflow library 
import sys
sys.path.append(r'modelflow/')
 
from modelsandbox import newmodel  
from modelnet     import draw_adjacency_matrix
from modeljupyter import inputwidget

# Prepare some stuff

In [ ]:
turbo = 0  # Determines if the the model is compiled or not 

# define af function which is in Eviews but not in Python
@jit("f8(b1,f8,f8)")
def recode(condition,yes,no):
    '''Function which recratetes the functionality of @recode from eviews ''' 
    return yes if condition else no

# Load the model and the data 
The model and the data has been prepared from the public packagde in a previous step. 

In [ ]:
basedf  = pd.read_pickle('frbus/data/baseline.pc')

In [ ]:
ffrbus  = open('frbus/model/ffrbusvar.txt','rt').read() 
# print(ffrbus)

# Create a model class instance

In [ ]:
mfrbus = newmodel(ffrbus,funks=[recode])

mfrbus.use_preorder = True  

# Run Baseline

In [ ]:
res = mfrbus(basedf,'2020q1','2025q4',antal= 500,relconv=0.000000001,
             conv='XGDPN',first_test=10,ldumpvar=True,ljit=turbo,silent=1,debug=0)

## Provide descriptions for the model  

In [ ]:
trans = {'XGDPN':'GDP real terms (XGDPN)','RFF':'Federal Funds Rate (RFF)','GFSRPN':'Federal Budget surplus',
        'RFFMIN':'Minimum Fed Funds'}


# Define an interactive input widget

In [ ]:
# Define the input widget 

slidedef = {'Initial taylor':                {'var':'RFFINTAY_AERR','value':0.0,'min':-3, 'max':3,'op':'+'},
            'Initial taylor inital impulse': {'var':'RFFINTAY_AERR','value':0.0,'min':-3, 'max':3,'op':'+impulse'},
            'Minimum federal funds':         {'var':'RFFMIN',       'value':0.0,'min':-20,'max':5,'op':'='       ,'step':1.0}
            }
radiodef = {'Monetary policy reaction':
            [['Inertial Taylor rule','DMPINTAY'],
             ['Taylor rule','DMPTAY'],
             ['Taylor rule with unemployment gap','DMPTLR'],
             ['Estimated rule','DMPALT'],
             ['Generalized rule','DMPGEN'],
             ['Exogenous nominal funds rate', 'DMPEX'],
             ['Exogenous real funds rate', 'DMPRR']
            ]
           }
           
checkdef = [('Timing of zero lower bound', 'DMPTRSH' ,True) ]

input = inputwidget(mfrbus,basedf,slidedef=slidedef,checkdef=checkdef,radiodef=radiodef
                   ,showout=True,varpat='RFF XGDPN RFFMIN GFSRPN DMPTRSH',modelopt={'silent':True},trans=trans)

# Run the widget  

In [ ]:
display(input)

# Some interfaces 
. (dot) makes it easy to look at single variable. 

\[ \] (Slice) alows inspection of many variables selected with wildcars

In [ ]:
 # we only want a few quarters 
_ = mfrbus.smpl('2020q1','2021q1')

In [ ]:
mfrbus.rff.show

In [ ]:
mfrbus['rffta* rffrule*']

In [ ]:
_= mfrbus.rffintay.dekomp(start='2020q1',end='2021q1')

In [ ]:
_ = mfrbus.rfftay.explain(up=1,svg=1,pdf=0)

In [ ]:
_ = mfrbus['rffta* rffrule*'].dif.plot(colrow=1,sharey=0)

In [ ]:
#mfrbus.drawendo(lag=0,title='FRB/US endogeneous interpendencies',pdf=0)
# you will find the output files in different format in the /graph folder

# How is the interest rate determined

In [ ]:
mfrbus.RFF.draw(up=1,down=1,HR=0,svg=1)

# Show causal structure (only current year)

In [ ]:
fig   = draw_adjacency_matrix(mfrbus.endograph,mfrbus.precoreepiorder,
                                          mfrbus._superstrongblock,mfrbus._superstrongtype,size=(40,40))
#fig.savefig('graph/FRB-US causality structure.pdf')